<a href="https://colab.research.google.com/github/Oleonn/DataMining/blob/Data-mining-Python/PyINaturalist_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importations et installations**
Executer les deux codes suivants

In [1]:
!pip install pyinaturalist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.5 MB/s eta 0:00:00


In [2]:
from pyinaturalist.v1.observations import get_observation_species_counts
import json
from pyinaturalist import (
    Observation,
    pprint,
    get_observations,
    get_observation_species_counts
)

import pandas as pd
pd.options.display.max_colwidth = 100  #Sert a augmenter la qte de caracteres affiches pour chaque string

import csv

import requests

import os

Pour importer une serie de photos de INaturalist, commencer par exporter les Occurences GBIF (en preselectionnant "INaturalist (research)" et "avec photo" dans les criteres de recherche) en format Archive Darwin Core. Donne un fichier Zip, qu'il faut ouvrir pour extraire "multimedia.txt". Ensuite, executer le code suivant pour lier le Drive de destination.

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Dans le menu "fichiers" a gauche, trouver le dossier destination et en copier le chemin d'acces en utilisant le sous-menu a droite du nom du fichier. 
Coller le chemin d'acces apres le "=" de "Coller_ici", et le mettre entre guillements. 
Ecrire ensuite le format d'image souhaite entre guillemets (du plus petit au plus grand : "thumb", "small", "medium", "large", "original"). 
Executer les codes restants.

In [5]:
Coller_ici = "/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis" #Exemple : "/content/drive/MyDrive/Projet_mellifere/Solidago_canadensis"
Format_souhaite = "small" #Exemple : "small"
Nb_images_souhaite = "toutes" #SOIT un nombre, SOIT "toutes" entre guillemets

In [6]:
Path = f"{Coller_ici}/"

In [7]:
Solcan = pd.read_csv(Path+"multimedia.txt", dtype=str, sep="\t")
Solcan = Solcan["identifier"]
if Nb_images_souhaite != "toutes":
  Solcan = Solcan[0:Nb_images_souhaite]
for Line in range(len(Solcan)):
  Solcan[Line] = Solcan[Line].replace("original", Format_souhaite) #Remplace le format original par le format souhaite (thumb, small, medium ou large)

In [8]:
#Solcan = Solcan[7501:] #Pendant retraite SBL, j'ai telecharge les images [0:1000]. Le 2 juin 2023, j'ai telecharge les images [1000:7501].

Sauvegarde en .jpg dans le Drive en utilisant l'ID de photo d'INaturalist. Attendre le message de confirmation comme quoi les telechargements ont ete accomplis.

In [ ]:
broken_images = []
count = 1
for img in Solcan:
    # We can split the file based on '/' and extract the last split within the Python list below:
    file_name = img.split('/')[-2]
    file_name = f"{Path}{file_name}.jpeg"  # Update file extension to .jpeg
    print(f"Fichier no {count} situe ici : {file_name}")
    count = count + 1
    # Now let's send a request to the image URL:
    r = requests.get(img, stream=True)
    # We can check that the status code is 200 before doing anything else:
    if r.status_code == 200:
        # This command below will allow us to write the data to a file as binary:
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)
    else:
        # We will write all of the images back to the broken_images list:
        broken_images.append(img)
      
print("Mission accomplie. Bon travail, 007")
